Using the dataset “wine_properties.csv” in the folder “Assignment_2” and Python, do the following tasks:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

• Perform a PCA, analysing the meaning of the first two principal components using the “circle of correlations”

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
wine = pd.read_csv('wines_properties.csv')
wine.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [4]:
n_factor = wine.shape[1] - 1
wine_std = StandardScaler().fit_transform(wine.iloc[:, :-1])

/home/ranger/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ranger/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [5]:
my_pca = PCA(n_components=2)
new_projected_data = my_pca.fit_transform(wine_std)
PCs = my_pca.components_

In [6]:
%matplotlib

fig = plt.figure(figsize=(5,5))
plt.quiver(np.zeros(PCs.shape[1]), np.zeros(PCs.shape[1]),
           PCs[0,:], PCs[1,:], 
           angles='xy', scale_units='xy', scale=1)

# Add labels based on feature names (here just numbers)
feature_names = np.arange(PCs.shape[1])
for i,j,z in zip(PCs[1,:]+0.02, PCs[0,:]+0.02, feature_names):
    plt.text(j, i, z, ha='center', va='center')

# Add unit circle
circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
plt.gca().add_artist(circle)

# Ensure correct aspect ratio and axis limits
plt.axis('equal')
plt.xlim([-1.0,1.0])
plt.ylim([-1.0,1.0])

# Label axes
plt.xlabel('PC 0')
plt.ylabel('PC 1')

# Done
plt.savefig('corr.png')
plt.show()

Using matplotlib backend: TkAgg


In [7]:
PC1_cor = np.zeros(n_factor)
for i in range(n_factor):
    PC1_cor[i] = np.corrcoef(wine_std.dot(PCs[0]), wine_std[:, i])[0, 1]
    
PC2_cor = np.zeros(n_factor)
for i in range(n_factor):
    PC2_cor[i] = np.corrcoef(wine_std.dot(PCs[1]), wine_std[:, i])[0, 1]

In [8]:
wine.iloc[:, :-1].columns[np.abs(PC1_cor) > .75]

Index(['Total_Phenols', 'Flavanoids', 'OD280'], dtype='object')

In [9]:
wine.iloc[:, :-1].columns[np.abs(PC2_cor) > .75]

Index(['Alcohol', 'Color_Intensity'], dtype='object')

PCs is factor loadings, the weight by which each standardized original factor should be multiplied to get the factor scores. The correlation coefficient between factor scores and standardized original factor produces the circle of correlations. According to the picture above:

Factor 'Total_Phenols', 'Flavanoids', 'OD280', are represented by PC0.  
Factor 'Alcohol', 'Color_Intensity' are represented by PC1.  
Fhe rest factors are evenly represented by PC0 & PC1.

• Use a hierarchical cluster algorithm to guess a likely number of cluster present in the data

In [10]:
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram

In [11]:
row_dist = pd.DataFrame(squareform(pdist(new_projected_data, metric='euclidean')))
row_dist.head()

,0,1,2,3,4,5,6,7,8,9,...,168,169,170,171,172,173,174,175,176,177
0,0.000000,2.093633,0.900010,1.384777,2.378064,0.729368,0.908289,1.268160,0.962015,0.863050,...,5.534605,5.811629,6.744749,7.019991,5.830787,6.731783,5.927018,6.137650,5.767329,6.658761
1,2.093633,0.000000,1.398713,3.455679,1.699730,2.595737,1.527160,1.948141,1.287249,1.247743,...,5.009231,5.441520,5.421704,6.004696,5.314369,6.134913,5.245996,5.784495,5.296078,6.243515
2,0.900010,1.398713,0.000000,2.124804,1.516437,1.214686,0.158827,0.736880,0.113232,0.338440,...,4.813270,5.139359,5.870147,6.197373,5.116153,6.005367,5.169936,5.475005,5.064591,5.983408
3,1.384777,3.455679,2.124804,0.000000,3.333377,0.949474,2.052319,2.049022,2.220888,2.208103,...,5.977152,6.140284,7.590015,7.676056,6.248000,7.148022,6.437037,6.434907,6.161206,6.965835
4,2.378064,1.699730,1.516437,3.333377,0.000000,2.395000,1.472127,1.284495,1.502740,1.746571,...,3.411261,3.800709,4.366882,4.686879,3.718171,4.581744,3.718308,4.143459,3.683763,4.625500


In [12]:
row_clusters = linkage(pdist(new_projected_data, metric='euclidean'), method='complete')
cluster_process = pd.DataFrame(row_clusters,
             columns=['row label 1', 'row label 2',
                      'distance', 'no. of items in clust.'],
             index=['cluster %d' % (i + 1) 
                    for i in range(row_clusters.shape[0])])

In [28]:
%matplotlib

row_dendr = dendrogram(row_clusters)
plt.tight_layout()
plt.ylabel('Euclidean distance')

Using matplotlib backend: TkAgg


Text(55.847222222222214, 0.5, 'Euclidean distance')

• Use the previous number of cluster to perform a K-means cluster analysis

In [14]:
from sklearn.cluster import KMeans

In [15]:
km = KMeans(n_clusters=3, 
            init='random', 
            n_init=10, 
            max_iter=300,
            tol=1e-04,
            random_state=0)

y_km = km.fit_predict(new_projected_data)

In [16]:
y_km

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int32)

In [17]:
np.array(wine.iloc[:, -1])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3])

In [18]:
y_km2 = y_km.copy()
for i in range(len(y_km2)):
    if y_km2[i] == 0:
        y_km2[i] = 1
    elif y_km2[i] == 1:
        y_km2[i] = 3
print('Accuracy of Prediction:', np.mean(y_km2 == np.array(wine.iloc[:, -1])))

Accuracy of Prediction: 0.9662921348314607


• Analyse the “silhouette” of the clusters

In [19]:
from sklearn.metrics import silhouette_samples
from matplotlib import cm

In [20]:
%matplotlib

cluster_labels = np.unique(y_km)
n_clusters = cluster_labels.shape[0]

silhouette_vals = silhouette_samples(new_projected_data, y_km, metric='euclidean')

y_ax_lower, y_ax_upper = 0, 0
yticks = []

for i, c in enumerate(cluster_labels):
    c_silhouette_vals = silhouette_vals[y_km == c]
    c_silhouette_vals.sort()
    y_ax_upper += len(c_silhouette_vals)
    color = cm.jet(float(i) / n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, 
             edgecolor='none', color=color)

    yticks.append((y_ax_lower + y_ax_upper) / 2.)
    y_ax_lower += len(c_silhouette_vals)
    
silhouette_avg = np.mean(silhouette_vals)
plt.axvline(silhouette_avg, color="red", linestyle="--") 

plt.yticks(yticks, cluster_labels + 1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')

plt.tight_layout()
plt.show()

Using matplotlib backend: TkAgg


• Plot on the space of the first two dimensions of the PCA the clusters obtained with K-means, using a different
colour for each cluster.

In [29]:
import seaborn as sns

In [30]:
%matplotlib

plot_data = pd.concat([ pd.DataFrame(new_projected_data, columns = ['PC0', 'PC1']), pd.DataFrame(y_km, columns = ['cluster']) ], axis = 1)
sns_plot = sns.relplot(x="PC0", y="PC1", hue='cluster', palette= 'Set1', data=plot_data)
sns_plot.savefig('cluster')

Using matplotlib backend: TkAgg


• For each cluster, which “original” variables (ex ante the PCA) are more important? Consider the barycenter of each cluster (the barycenter is an observation) and its variables values.

In [31]:
data_with_cluster = pd.concat([ pd.DataFrame(wine_std, columns = wine.iloc[:, :-1].columns), pd.DataFrame(y_km, columns = ['cluster'])], axis = 1)
data_with_cluster.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,cluster
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009,0
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242,0
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148,0
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574,0
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874,0


In [32]:
geo_center = pd.DataFrame(wine_std).groupby(y_km).mean()
geo_center.columns = wine.iloc[:, :-1].columns
geo_center

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline
0,0.847519,-0.296483,0.386655,-0.600529,0.589060,0.896994,0.983798,-0.560492,0.596144,0.183498,0.465747,0.790208,1.129373
1,0.177115,0.906507,0.215969,0.551040,-0.077345,-0.990100,-1.227118,0.713487,-0.761279,0.954384,-1.190063,-1.289398,-0.396321
2,-0.903797,-0.406566,-0.513201,0.135526,-0.478588,-0.077785,0.020063,-0.022155,0.025360,-0.879292,0.464068,0.242794,-0.732474


In [25]:
cluster_data = data_with_cluster.groupby('cluster')

rep_id_ls = cluster_data \
    .apply(pdist, metric='euclidean') \
    .apply(squareform)\
    .apply(pd.DataFrame) \
    .apply(lambda x: pd.concat([x, pd.DataFrame(x.sum(axis = 1), columns = ['sum'])] , axis = 1)) \
    .apply(lambda x: x['sum'].idxmin()) \
    .tolist()
rep_id_ls = np.array([cluster_data.get_group(i).iloc[idx: idx+1].index.values for i, idx in zip(range(len(rep_id_ls)), rep_id_ls)]).squeeze()

In [26]:
barycenter = data_with_cluster.loc[rep_id_ls]
barycenter

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,cluster
48,1.358028,-0.283974,0.122392,-0.208681,0.228785,0.728881,0.894264,-0.337251,1.382572,0.493956,0.493797,0.195361,0.997086,0
148,0.394521,0.811175,0.049285,0.602088,-0.543562,-0.585031,-1.274305,0.710264,-0.597284,1.454261,-1.787619,-1.400699,-0.308556,1
106,-0.927212,-0.544297,-0.901103,-0.148624,-1.386122,-1.033684,0.000733,0.065639,0.068508,-0.717240,0.186684,0.788587,-0.754385,2


In [27]:
for index, row in barycenter.iterrows():
    print('cluster ' + str(int(row['cluster'])) + ' :' + abs(row[: -1]).idxmax())

cluster 0 :Proanthocyanins
cluster 1 :Hue
cluster 2 :Magnesium


• Using both the information of barycenters and of PCA, give an interpretation to each cluster

Magnesium is more important to cluster 0. This cluster scores low in both PCs.  
Proanthocyanins is more important to cluster 1. This cluster socres high in PC1 and around 0 in PC0.  
Hue is more important to cluster 2. This cluster scores high in PC0 but low in PC1.

• Write a function that takes in input the dataset and that returns 1) the value of K (for the K-means) that is associated
with the best overall silhouette of the K-means algorithm and 2) the plot of the correspondent clusters on the space of
the first two dimensions of the PCA (performed over the same dataset). 

• Write a function that takes in input the dataset: the function performs the PCA and returns the circle of correlations of
each pair of principal components (1 and 2, 1 and 3, 1 and …, 2 and 1, 2 and 3, …). Plot all the circles in the same plot
and/or in a series of plots 3x3.

In [28]:
def process(data):
    from scipy.spatial.distance import pdist
    from scipy.cluster.hierarchy import linkage, cut_tree
    from sklearn.metrics import silhouette_samples
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    import numpy as np
    import seaborn as sns
    %matplotlib
    
    n_sample = data.shape[0]
    n_pc = 2
    my_pca = PCA(n_components=n_pc)
    data_std = StandardScaler().fit_transform(data)
    new_projected_data = my_pca.fit_transform(data_std)
    PCs = my_pca.components_
    row_clusters = linkage(pdist(new_projected_data, metric='euclidean'), method='complete')
    best_n_cluster = 1
    silhouette = -1
    for n_cluster in np.arange(2, n_sample - 1):
        silhouette_new = silhouette_samples(new_projected_data, cut_tree(row_clusters, n_cluster).squeeze(), metric='euclidean').mean()
        if silhouette_new > silhouette:
            silhouette = silhouette_new
            best_n_cluster = n_cluster

    km = KMeans(n_clusters=best_n_cluster, 
                init='random', 
                n_init=10, 
                max_iter=300,
                tol=1e-04,
                random_state=0)
    y_km = km.fit_predict(new_projected_data)

    plot_data = pd.concat([ pd.DataFrame(new_projected_data, columns = ['PC0', 'PC1']), pd.DataFrame(y_km, columns = ['cluster']) ], axis = 1)
    sns_plot = sns.relplot(x="PC0", y="PC1", hue='cluster', palette= 'Set1', data=plot_data)
    return (best_n_cluster, sns_plot)


In [29]:
best_n_cluster, sns_plot = process(wine)

Using matplotlib backend: TkAgg


/home/ranger/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ranger/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
best_n_cluster

In [ ]:
sns_plot.fig

In [35]:
def process2(data):
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    %matplotlib
    
    n_pc = data.shape[1]
    my_pca = PCA(n_components=n_pc)
    data_std = StandardScaler().fit_transform(data)
    new_projected_data = my_pca.fit_transform(data_std)
    PCs = my_pca.components_
    plt.figure(figsize=(5*n_pc, 5*n_pc))
    n_pc = data.shape[1]
    for s in range(n_pc):
            for t in range(n_pc):
                if s == t:
                    continue
                plt.subplot(n_pc, n_pc, s * n_pc + t + 1)
                plt.quiver(np.zeros(PCs.shape[1]), np.zeros(PCs.shape[1]),
                           PCs[s,:], PCs[t,:], 
                           angles='xy', scale_units='xy', scale=1)

                # Add labels based on feature names (here just numbers)
                feature_names = np.arange(PCs.shape[1])
                for i,j,z in zip(PCs[t,:]+0.02, PCs[s,:]+0.02, feature_names):
                    plt.text(j, i, z, ha='center', va='center')

                # Add unit circle
                circle = plt.Circle((0,0), 1, facecolor='none', edgecolor='b')
                plt.gca().add_artist(circle)

                # Ensure correct aspect ratio and axis limits
                plt.axis('equal')
                plt.xlim([-1,1])
                plt.ylim([-1,1])
    
                # Label axes
                plt.xlabel('PC 0')
                plt.ylabel('PC 1')  

In [36]:
process2(wine)

Using matplotlib backend: TkAgg


/home/ranger/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/ranger/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
